# 영화 추천 시스템 만들기

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

# MovieLens 1M Dataset 불러오기

### 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


### 별점을 시청횟수로 해석해서 생각하겠습니다.

In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts'].head()

0    5
1    3
2    3
3    4
4    5
Name: counts, dtype: int64

### timestamp은 영화 시청 날짜를 나타내므로 필요가 없다.

In [6]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id', 'counts']
ratings = ratings[using_cols]
ratings.head()

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### 검색하기 쉽게 소문자로 변경

In [8]:
movies['title'] = movies['title'].str.lower()
movies['genre'] = movies['genre'].str.lower()

In [9]:
movies.head()

,movie_id,title,genre
0,1,toy story (1995),animation|children's|comedy
1,2,jumanji (1995),adventure|children's|fantasy
2,3,grumpier old men (1995),comedy|romance
3,4,waiting to exhale (1995),comedy|drama
4,5,father of the bride part ii (1995),comedy


# 데이터 분석하기

### ratings에 있는 유니크한 영화 개수, ratings에 있는 유니크한 사용자 수

In [10]:
# user_id의 개수와 movie_id의 개수 확인
print(ratings["movie_id"].nunique())
print(ratings["user_id"].nunique())

3628
6039


In [11]:
print('movue_id 결측치 확인 :', ratings["movie_id"].isna().sum())
print('user_id 결측치 확인  :', ratings["user_id"].isna().sum())
print('counts 결측치 확인   :', ratings["counts"].isna().sum())

movue_id 결측치 확인 : 0
user_id 결측치 확인  : 0
counts 결측치 확인   : 0


In [12]:
print('movue_id 결측치 확인 :', movies["movie_id"].isna().sum())
print('user_id 결측치 확인  :', movies["title"].isna().sum())
print('counts 결측치 확인   :', movies["genre"].isna().sum())

movue_id 결측치 확인 : 0
user_id 결측치 확인  : 0
counts 결측치 확인   : 0


In [13]:
#ratings와 movies를 이어붙임 = 원활한 데이터 분석을 위해
data = pd.merge(movies, ratings)
data.head()

,movie_id,title,genre,user_id,counts
0,1,toy story (1995),animation|children's|comedy,1,5
1,1,toy story (1995),animation|children's|comedy,6,4
2,1,toy story (1995),animation|children's|comedy,8,4
3,1,toy story (1995),animation|children's|comedy,9,5
4,1,toy story (1995),animation|children's|comedy,10,5


### 가장 인기 있는 영화 30개(인기순)

In [14]:
#인기있는 영화순
movie_count = data.groupby("title")["user_id"].count()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [15]:
# 유저별 몇 편의 영화를 보고 평가 했는지에 대한 통계
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

# 내가 선호하는 영화 추가하기

In [16]:
#내가 선호하는 5가지 영화
my_favorite = ["sixth sense, the (1999)", "men in black (1997)", "terminator 2: judgment day (1991)", "toy story (1995)", "terminator, the (1984)"]

# yyttt라는 user_id가 위 영화를 5점씩 주었다고 가정 # 이 정보를 바탕으로 추천받기위해
my_playlist = pd.DataFrame({'user_id': ['gomen']*5, 'title': my_favorite, 'counts':[5]*5})

# user_id에 'yyttt'라는 데이터가 없다면
# 위에 임의로 만든 my_favorite 데이터를 추가         
if not data.isin({"user_id":["gomen"]})["user_id"].any():
    data = data.append(my_playlist)
    
data.tail()

,movie_id,title,genre,user_id,counts
0,NaN,"sixth sense, the (1999)",NaN,gomen,5
1,NaN,men in black (1997),NaN,gomen,5
2,NaN,terminator 2: judgment day (1991),NaN,gomen,5
3,NaN,toy story (1995),NaN,gomen,5
4,NaN,"terminator, the (1984)",NaN,gomen,5


In [17]:
data[data['user_id']=='gomen']

,movie_id,title,genre,user_id,counts
0,NaN,"sixth sense, the (1999)",NaN,gomen,5
1,NaN,men in black (1997),NaN,gomen,5
2,NaN,terminator 2: judgment day (1991),NaN,gomen,5
3,NaN,toy story (1995),NaN,gomen,5
4,NaN,"terminator, the (1984)",NaN,gomen,5


In [18]:
data = data.drop(['movie_id','genre'], axis=1)

In [19]:
data.tail(10)

,title,user_id,counts
836473,"contender, the (2000)",5682,3
836474,"contender, the (2000)",5812,4
836475,"contender, the (2000)",5831,3
836476,"contender, the (2000)",5837,4
836477,"contender, the (2000)",5998,4
0,"sixth sense, the (1999)",gomen,5
1,men in black (1997),gomen,5
2,terminator 2: judgment day (1991),gomen,5
3,toy story (1995),gomen,5
4,"terminator, the (1984)",gomen,5


In [20]:
data.reset_index()

,index,title,user_id,counts
0,0,toy story (1995),1,5
1,1,toy story (1995),6,4
2,2,toy story (1995),8,4
3,3,toy story (1995),9,5
4,4,toy story (1995),10,5
...,...,...,...,...
836478,0,"sixth sense, the (1999)",gomen,5
836479,1,men in black (1997),gomen,5
836480,2,terminator 2: judgment day (1991),gomen,5
836481,3,toy story (1995),gomen,5


In [21]:
# 고유한 유저, title을 찾아내는 코드
user_unique = data["user_id"].unique()
title_unique = data["title"].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [22]:
print(len(user_unique))
print(len(title_unique))

6040
3628


In [23]:
print(user_to_idx['gomen'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039가 나와야 함
print(title_to_idx["men in black (1997)"])

6039
1419


In [26]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing Fail!!
title column indexing OK!!


,title,user_id,counts
0,0,0,5
1,0,1,4
2,0,2,4
3,0,3,5
4,0,4,5
...,...,...,...
0,2507,6039,5
1,1419,6039,5
2,569,6039,5
3,0,6039,5


# CSR matrix 만들기

In [27]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# AlternatingLeastSquares 모델 구성

In [28]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [29]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=400, regularization=0.01, use_gpu=False, iterations=200, dtype=np.float32)

In [30]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [31]:
#훈련시작
als_model.fit(csr_data_transpose)

  0%|          | 0/200 [00:00<?, ?it/s]

# 모델 예측하기

In [32]:
gomen, men_in_black = user_to_idx['gomen'], title_to_idx['men in black (1997)']
gomen_vector, men_in_black_vector = als_model.user_factors[gomen], als_model.item_factors[men_in_black]

In [33]:
gomen_vector

array([-2.99138087e-03,  3.45750749e-01, -4.70748730e-02, -2.95501232e-01,
        4.54890132e-02, -1.01190001e-01, -6.61127716e-02,  1.60814270e-01,
        2.10321397e-01, -1.63021926e-02, -5.57704922e-03, -6.76446259e-02,
        1.78148612e-01, -2.44523715e-02,  1.70569479e-01, -1.35325696e-02,
       -5.08850552e-02,  9.07529145e-02,  1.68547451e-01, -1.68830194e-02,
       -3.94071899e-02,  1.58807158e-01, -7.65735330e-03, -1.19559847e-01,
       -4.69308458e-02,  1.31576091e-01, -2.22216919e-01, -4.13275547e-02,
        1.95156306e-01, -6.40356764e-02,  7.18981773e-02, -7.81773031e-02,
        1.41591756e-02, -8.08097944e-02, -5.88849261e-02,  9.58070010e-02,
       -8.84692892e-02, -2.70925671e-01,  3.39462645e-02,  1.81657672e-01,
        1.39633417e-01,  6.79516345e-02, -5.65957278e-02,  7.28564337e-02,
       -6.45353422e-02, -8.93636942e-02, -9.84376594e-02, -2.77386084e-02,
        1.80333350e-02, -1.68084472e-01,  1.63027287e-01, -1.21327445e-01,
       -8.33841041e-02,  

In [34]:
men_in_black_vector

array([ 3.97901684e-02,  1.44497011e-04,  3.38366081e-04, -9.96457972e-03,
       -3.67158987e-02, -1.97237302e-02,  1.09891687e-03,  1.30282082e-02,
        3.09972633e-02,  3.78560051e-02, -2.28744186e-02,  4.07522684e-03,
        4.44239192e-02,  1.46804657e-02,  4.46226671e-02,  2.52974145e-02,
        1.29544865e-02,  3.48298065e-02,  2.77267415e-02,  2.21979134e-02,
       -7.22251609e-02,  3.88558470e-02, -1.54822040e-02, -2.72834711e-02,
       -5.51426634e-02, -1.21324398e-02,  4.26587239e-02,  4.36684750e-02,
        3.72486562e-02, -5.54168113e-02, -5.98321157e-03,  2.26331577e-02,
       -6.80039674e-02,  1.21453160e-03, -4.02473025e-02,  3.53895314e-02,
       -2.09448021e-02, -7.51169398e-03,  2.27685533e-02,  3.72727290e-02,
       -6.66618580e-03, -2.17274274e-03,  3.24617773e-02,  2.26346892e-03,
        3.03186458e-02, -5.73004484e-02,  5.32735186e-03, -4.21667174e-02,
        3.68364155e-02, -9.54430725e-04,  8.10692757e-02, -3.45356129e-02,
       -2.01739576e-02,  

In [35]:
# gomen, men_in_black을 내적하는 코드
np.dot(gomen_vector, men_in_black_vector)

0.88910604

In [37]:
sixth_sense = title_to_idx['sixth sense, the (1999)']
sixth_sense_vector = als_model.item_factors[sixth_sense]
np.dot(gomen_vector, sixth_sense_vector)

0.91976947

In [38]:
idx_to_title = {v:k for k,v in title_to_idx.items()}

def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [39]:
get_similar_title("toy story (1995)")

['toy story (1995)',
 'toy story 2 (1999)',
 'aladdin (1992)',
 'soft toilet seats (1999)',
 "bug's life, a (1998)",
 'nobody loves me (keiner liebt mich) (1994)',
 'lion king, the (1994)',
 'babe (1995)',
 'groundhog day (1993)',
 'best men (1997)']

In [40]:
get_similar_title("contender, the (2000)")

['contender, the (2000)',
 'nurse betty (2000)',
 'almost famous (2000)',
 'wonder boys (2000)',
 'best in show (2000)',
 'requiem for a dream (2000)',
 'yards, the (1999)',
 'tigerland (2000)',
 'remember the titans (2000)',
 'watcher, the (2000)']

In [41]:
get_similar_title("forrest gump (1994)")

['forrest gump (1994)',
 "devil's brigade, the (1968)",
 'outside ozona (1998)',
 "child's play 3 (1992)",
 'soft fruit (1999)',
 'digimon: the movie (2000)',
 'anna (1996)',
 'death in the garden (mort en ce jardin, la) (1956)',
 'pretty woman (1990)',
 'ulysses (ulisse) (1954)']

# 가장 선호하는 영화 추천 받기

In [44]:
user = user_to_idx['gomen']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
title_recommended

[(2845, 0.1850586),
 (2325, 0.17934443),
 (1084, 0.16045839),
 (2657, 0.124851026),
 (949, 0.1235615),
 (1369, 0.12164909),
 (462, 0.11260992),
 (1098, 0.107155286),
 (20, 0.10388384),
 (439, 0.10298101),
 (2542, 0.09958276),
 (2903, 0.09601663),
 (574, 0.095674485),
 (2667, 0.089631446),
 (3127, 0.08799505),
 (1245, 0.08765642),
 (995, 0.08628108),
 (3224, 0.08271313),
 (1160, 0.08133225),
 (33, 0.081058696)]

In [45]:
[idx_to_title[i[0]] for i in title_recommended]

['toy story 2 (1999)',
 'matrix, the (1999)',
 'aliens (1986)',
 'total recall (1990)',
 'die hard (1988)',
 'austin powers: international man of mystery (1997)',
 'jurassic park (1993)',
 'alien (1979)',
 'get shorty (1995)',
 'fugitive, the (1993)',
 'big (1988)',
 'talented mr. ripley, the (1999)',
 'snow white and the seven dwarfs (1937)',
 'hairspray (1988)',
 'jungle fever (1991)',
 'star trek: the motion picture (1979)',
 'reservoir dogs (1992)',
 'predator (1987)',
 'young frankenstein (1974)',
 'babe (1995)']

In [50]:
matrix_trousers = title_to_idx["matrix, the (1999)"]
explain = als_model.explain(user, csr_data, itemid=matrix_trousers)

In [51]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('terminator 2: judgment day (1991)', 0.10647738830041029),
 ('sixth sense, the (1999)', 0.04027354363577505),
 ('toy story (1995)', 0.02811144939117893),
 ('men in black (1997)', 0.008415775436988538),
 ('terminator, the (1984)', -0.004120889583034762)]

In [56]:
movies[movies['title']=='matrix, the (1999)']

,movie_id,title,genre
2502,2571,"matrix, the (1999)",action|sci-fi|thriller


In [57]:
movies[movies['title']=='terminator 2: judgment day (1991)']

,movie_id,title,genre
585,589,terminator 2: judgment day (1991),action|sci-fi|thriller


In [58]:
movies[movies['title']=='terminator, the (1984)']

,movie_id,title,genre
1220,1240,"terminator, the (1984)",action|sci-fi|thriller


In [59]:
movies[movies['title']=='toy story (1995)']

,movie_id,title,genre
0,1,toy story (1995),animation|children's|comedy


# 회고

셀을 합치는 함수로 concat()을 사용했으나 아무런 옵션 없이 행 인덱스와 열 인덱스가 서로 다른 두 데이터프레임을 병합하는 경우 위와 같이 병합되기 전 자신의 인덱스를 그대로 사용하고 열 인덱스가 맞지 않는 부분은 NaN 값으로 채워진다. 그래서 키값을 기준으로 병합을 하는 merge() 함수를 사용해서 문제를 해결할 수 있었습니다. MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정된 것을 확인할 수 있었습니다. toy story를 검색했을 때 알맞은 영화를 추천해주는 것을 볼 수 있었습니다. 
매트릭스 영화를 추천한데 터미네이터2가 많은 기여를 한 것을 확인했다. matrix, the (1999) 장르와 영향을 가장 많이준 terminator 2: judgment day (1991)의 장르가 일치하는 것을 보아 장르가 추천에 기여를 하는 것 같습니다. 그러나 toy story (1995)의 장르는 다릅니다. 그래서 장르가 무조건 적으로 영향을 끼친다고는 할 수 없습니다.